In [ ]:
import sys
sys.path.append('../support')
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
from astropy.cosmology import FlatLambdaCDM
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
from sampler import fitters
from astropy.coordinates import SkyCoord
from astropy.table import Table
import GCRCatalogs



Make sure we know which version we're using

In [ ]:
clmm.__version__

In [ ]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)

#### Read in the extragalactic catalog

In [ ]:
extragalactic_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

#### Get the list of halos with `M > mmin` in the redshift range `[zmin, zmax]`

In [ ]:
# get list of massive halos in a given redshift and mass range
mmin = 3.e14
zmin = 0.3
zmax = 0.4

massive_halos = extragalactic_cat.get_quantities(['halo_mass', 'redshift','ra', 'dec'],\
                                                 filters=[f'halo_mass > {mmin}','is_central==True',
                                                          f'redshift>{zmin}', f'redshift<{zmax}'])


In [ ]:
N_cl = len(massive_halos['halo_mass'])
print(f'There are {N_cl} clusters available')

# Selecting the most massive one
select = massive_halos['halo_mass'] == np.max(massive_halos['halo_mass'])

In [ ]:
ra_cl = massive_halos['ra'][select][0]
dec_cl = massive_halos['dec'][select][0]
z_cl = massive_halos['redshift'][select][0]
mass_cl =massive_halos['halo_mass'][select][0]
print (f'The most massive cluster is in z = {z_cl:.2f}, with mass = {mass_cl:.2e} Msun')

### Apply coordinates, redshift and magnitude cuts to select backgroud galaxies around the cluster

In [ ]:
# get galaxies around it
ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
z_min = z_cl + 0.1
mag_i_max = 25

coord_filters = ['ra >= {}'.format(ra_min),'ra < {}'.format(ra_max),'dec >= {}'.format(dec_min),'dec < {}'.format(dec_max)]
z_filters = ['redshift >= {}'.format(z_min)]
mag_filters = ['mag_i < {}'.format(mag_i_max)]
gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2','redshift', 'convergence'],\
                                           filters=(coord_filters + z_filters + mag_filters))



In [ ]:
e1, e2 = clmm.utils.convert_shapes_to_epsilon(gal_cat['shear_1'],gal_cat['shear_2'],\
                                              shape_definition='shear',kappa=gal_cat['convergence'])

#store the results into an astropy table
dat = Table([gal_cat['ra'],gal_cat['dec'],e1,\
      e2,gal_cat['redshift'],gal_cat['galaxy_id']], \
      names=('ra','dec', 'e1', 'e2', 'z','id'))

cl = clmm.GalaxyCluster('CL', ra_cl, dec_cl, z_cl, dat)   

In [ ]:
plt.hist(cl.galcat['z'], bins=30)

### Computes the tangential and cross shear profiles

#### Default binning

In [ ]:
bin_edges = clmm.polaraveraging.make_bins(0.15, 10, 20, method='evenlog10width')
cl.compute_shear(geometry="flat")
profile = cl.make_shear_profile("radians", "Mpc", bins=bin_edges,cosmo=cosmo)

### Set up a model

In [ ]:
reduced_shear_model = np.zeros(len(profile['radius']))

for i,r in enumerate(profile['radius']): 
    reduced_shear = clmm.predict_reduced_tangential_shear(r*cosmo.h, mass_cl*cosmo.h, 4.,z_cl, profile['z'], cosmo,
                                                        delta_mdef=200,halo_profile_model='nfw')  
    reduced_shear_model[i] = np.mean(reduced_shear)    

In [ ]:
plt.errorbar(profile['radius'],profile['gt'],yerr=profile['gt_err'])
plt.plot(profile['radius'],reduced_shear_model, label='model shear')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('R (Mpc)')
plt.ylabel('<shear>')